In [51]:
import numpy as np
import gensim 
from gensim.models.word2vec import Word2Vec 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC

import time

import helpers as HL
import cleaning as CL

In [2]:
filenames = ['train_pos.txt', 'train_neg.txt'] 

corpus, file_lengths = HL.create_corpus(filenames)

In [30]:
cluster_file_path = '50mpaths2.txt'
cluster_dict = CL.create_dictionary(cluster_file_path)

clusterised_corpus = CL.create_clusterized_corpus(corpus, cluster_dict)

In [4]:
class IterableCorpus(object):
    
    def __init__(self, corpus):
        self.corpus = corpus
        
    def __iter__(self):
        for tweet in corpus:
            tweet_words = tweet.split()
            yield [word.decode('utf-8') for word in tweet_words]

In [7]:
iterable_corpus = IterableCorpus(clusterised_corpus)

# SMÆKK INN HYPERPARAMETERE HER!!! 
model = Word2Vec(iterable_corpus)

In [24]:
#Just for fun
model.similarity('lol','haha')

0.70570331580231127

#### So we have our word2vec model, giving us a representation for each word in the whole corpus. We now need to create a representation of each Tweet in the dataset. 

We will do this with the following steps: 
- Creating a TFIDF model for each word
- Combining the word2vec-vectors for each word, weighting them by their TFIDF score
- Run that shit through a neural network fuck yeaaaaah

In [46]:
#Creating a vectorizer to use for Tf-IDF analysis on words
vectorizer = TfidfVectorizer(
        min_df = 6, # removing word that occure less then 10 times 
        max_df = 1.3, # remove words that are too frequent ( more then 0.8 * number of tweets )
        sublinear_tf=True, # scale the term frequency in logarithmic scale
        use_idf =True
        #stop_words = custom_stop_words # Removing stop-words
    )

In [42]:
#Training-set numbers
nr_of_positive_training_examples = 100000
nr_of_training_examples = 200000

#Making labels
labels = np.zeros(nr_of_training_examples);
labels[0:nr_of_positive_training_examples]=1;
labels[nr_of_positive_training_examples:nr_of_training_examples]=0; 

In [ ]:
corpus_tf_idf = vectorizer.fit_transform(corpus)


In [52]:
#THIS THING SUCKS, NO POINT IN USING THIS SHIT

kf = StratifiedKFold(n_splits=3)

best_score = 0
best_min_df = 0 
best_max_df = 0

min_df_list = list(range(4,13))
max_df_list = np.linspace(0.1, 2.0, num=15)

run = 0

for min_df in min_df_list:
    for max_df in max_df_list:
        
        start = time.time()

        totalacc = 0   # Accuracy measure 

        for train_index, test_index in kf.split(clusterised_corpus, labels):

            X_train = [corpus[i] for i in train_index]
            X_test = [corpus[i] for i in test_index]

            y_train, y_test = labels[train_index], labels[test_index]

            train_corpus_tf_idf = vectorizer.fit_transform(X_train) 
            test_corpus_tf_idf = vectorizer.transform(X_test)

            model1 = LinearSVC()
            model1.fit(train_corpus_tf_idf,y_train)
            result1 = model1.predict(test_corpus_tf_idf)

            totalacc = totalacc+sum(y_test==result1)

        score = totalacc/nr_of_training_examples  

        if(score > best_score):
            best_min_df = min_df
            best_max_df = max_df
            best_score = score

            print("Best score: ", best_score, "\n",
                  "With min_df:", min_df, " and ",
                  "Max_df:", max_df)

        elapsed = time.time()
        print("Run: ", run, "- Time:", elapsed-start)
        run += 1

Best score:  0.792595 
 With min_df: 4  and  Max_df: 0.1
Run:  0 - Time: 20.833454370498657
Run:  1 - Time: 17.848948001861572
Run:  2 - Time: 16.260960578918457
Run:  3 - Time: 16.54555106163025
Run:  4 - Time: 16.23372721672058
Run:  5 - Time: 16.393404722213745
Run:  6 - Time: 18.04198408126831
Run:  7 - Time: 17.269551038742065
Run:  8 - Time: 16.61327600479126
Run:  9 - Time: 16.88491654396057
Run:  10 - Time: 18.763741970062256
Run:  11 - Time: 17.548179149627686
Run:  12 - Time: 23.25485873222351
Run:  13 - Time: 23.963690042495728
Run:  14 - Time: 22.57299041748047
Run:  15 - Time: 16.734185934066772
Run:  16 - Time: 16.680673837661743
Run:  17 - Time: 18.005351066589355
Run:  18 - Time: 17.146661520004272
Run:  19 - Time: 16.9967257976532
Run:  20 - Time: 16.815205097198486
Run:  21 - Time: 17.049448013305664
Run:  22 - Time: 16.200737714767456
Run:  23 - Time: 16.63927698135376
Run:  24 - Time: 18.22055983543396
Run:  25 - Time: 17.51658821105957
Run:  26 - Time: 17.334141016